# SQL исследование данных об инвестиции венчурных фондов в компании-стартапы

Посчитайте, сколько компаний закрылось.

In [ ]:
SELECT COUNT(status)
FROM company
WHERE status = 'closed';


Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total .

In [ ]:
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC;

Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

In [ ]:
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM CAST(acquired_at AS date)) BETWEEN '2011' AND '2013';

Отобразите имя, фамилию и названия аккаунтов людей в твиттере, у которых названия аккаунтов начинаются на 'Silver'.

In [ ]:
SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

Выведите на экран всю информацию о людях, у которых названия аккаунтов в твиттере содержат подстроку 'money', а фамилия начинается на 'K'.

In [ ]:
SELECT * 
FROM people
WHERE twitter_username LIKE '%money%' 
AND last_name LIKE 'K%';

Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

In [ ]:
SELECT country_code,
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

In [ ]:
SELECT funded_at,
       MIN(raised_amount),
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) <> 0 AND MIN(raised_amount) <> MAX(raised_amount);

Создайте поле с категориями:
Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
Отобразите все поля таблицы fund и новое поле с категориями.

In [ ]:
SELECT *,
       CASE 
           WHEN invested_companies >= 100 THEN 'high_activity'
           WHEN invested_companies >= 20 and invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies < 20 THEN 'low_activity'
       END
FROM fund;

Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

In [ ]:
SELECT i.activity,
       ROUND(AVG(investment_rounds))
FROM
(SELECT *,
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity
FROM fund) AS i
GROUP BY i.activity
ORDER BY ROUND(AVG(investment_rounds));

Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

In [ ]:
SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS DATE)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC
LIMIT 10;

Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

In [ ]:
SELECT a.first_name,
       a.last_name,
       b.instituition
FROM people AS a
LEFT OUTER JOIN education AS b ON a.id = b.person_id;

Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

In [ ]:
SELECT a.name,
       COUNT(DISTINCT c.instituition)
FROM company AS a
JOIN people AS b ON a.id = b.company_id
JOIN education AS c ON b.id = c.person_id
GROUP BY a.name
ORDER BY COUNT(DISTINCT c.instituition) DESC
LIMIT 5;

Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

In [ ]:
SELECT DISTINCT name
FROM company 
WHERE id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed';

Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

In [ ]:
SELECT id
FROM people
WHERE company_id IN
(SELECT id
FROM company 
WHERE id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed');

Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

In [ ]:
SELECT DISTINCT person_id,
       instituition
FROM education
WHERE person_id IN
(SELECT id
FROM people
WHERE company_id IN
(SELECT id
FROM company 
WHERE id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed'));

Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

In [ ]:
SELECT person_id,
       COUNT(instituition)
FROM education
WHERE person_id IN
(SELECT id
FROM people
WHERE company_id IN
(SELECT id
FROM company 
WHERE id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed'))
GROUP BY person_id;

Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

In [ ]:
SELECT AVG(all_info.all_instituition)
FROM
(SELECT person_id,
       COUNT(instituition) AS all_instituition
FROM education
WHERE person_id IN
(SELECT id
FROM people
WHERE company_id IN
(SELECT id
FROM company 
WHERE id IN (SELECT company_id
             FROM funding_round
             WHERE is_first_round = 1 AND is_last_round = 1)
AND status = 'closed'))

Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.

In [ ]:
SELECT AVG(mid_instituition2.mid)
FROM
(SELECT mid_instituition.person_id,
       COUNT(mid_instituition.instituition) AS mid
FROM
(SELECT person_id,
       instituition
FROM education
WHERE person_id IN
(SELECT id
FROM people 
WHERE company_id IN (SELECT id
             FROM company
             WHERE name = 'Facebook'))) AS mid_instituition
             GROUP BY mid_instituition.person_id) AS mid_instituition2;

Составьте таблицу из полей:
name_of_fund — название фонда;
name_of_company — название компании;
amount — сумма инвестиций, которую привлекла компания в раунде.
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund, 
       c.name AS name_of_company, 
         fr.raised_amount AS amount
FROM investment AS inv 
LEFT JOIN company AS c ON c.id = inv.company_id
LEFT JOIN fund AS f ON f.id = inv.fund_id
INNER JOIN (SELECT * 
           FROM funding_round
           WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2012 AND 2013) AS fr ON fr.id = inv.funding_round_id
WHERE c.milestones > 6;

Выгрузите таблицу, в которой будут такие поля:
название компании-покупателя;
сумма сделки;
название компании, которую купили;
сумма инвестиций, вложенных в купленную компанию;
доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c1.name AS name1, 
       ac.price_amount,
       c2.name AS name2, 
       c2.funding_total,
       ROUND(ac.price_amount / c2.funding_total)
FROM acquisition AS ac
LEFT JOIN company AS c1 ON ac.acquiring_company_id=c1.id
LEFT JOIN company AS c2 ON ac.acquired_company_id=c2.id
 
WHERE ac.price_amount > 0 AND c2.funding_total>0
ORDER BY ac.price_amount DESC, name2 
LIMIT 10;

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name,
      EXTRACT(MONTH FROM CAST(fr.funded_at AS date))
FROM funding_round AS fr
LEFT JOIN company AS c ON c.id = fr.company_id
WHERE c.category_code = 'social' AND EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) 
BETWEEN '2010' AND '2013' AND fr.raised_amount > 0;

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
номер месяца, в котором проходили раунды;
количество уникальных названий фондов из США, которые инвестировали в этом месяце;
количество компаний, купленных за этот месяц;
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH USA AS (SELECT EXTRACT(MONTH FROM fr.funded_at) AS month_number,
                           COUNT(DISTINCT f.name) AS usa_funds
                    FROM funding_round AS fr 
                    LEFT JOIN investment AS inv ON inv.funding_round_id = fr.id
                    LEFT JOIN fund AS f ON inv.fund_id = f.id
                    WHERE EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013 
                     AND f.country_code = 'USA'
                    GROUP BY month_number),
    comp AS (SELECT EXTRACT(MONTH FROM acquired_at) AS month_number,
                              COUNT(acquired_company_id) AS company_number,
                              SUM(price_amount) AS total_sum
                       FROM acquisition
                       WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013 
                       GROUP BY month_number)
SELECT USA.month_number,
       USA.usa_funds,
         comp.company_number,
         comp.total_sum
FROM USA JOIN comp ON USA.month_number = comp.month_number;

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH
i AS (SELECT country_code,
       AVG(funding_total) AS AVG2011
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
GROUP BY country_code),
i2 AS (SELECT country_code,
       AVG(funding_total) AS AVG2012
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
GROUP BY country_code),
i3 AS (SELECT country_code,
       AVG(funding_total) AS AVG2013
FROM company
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
GROUP BY country_code)

SELECT i.country_code,
       i.AVG2011,
       i2.AVG2012,
       i3.AVG2013
FROM  i JOIN i2 ON  i.country_code = i2.country_code JOIN i3 ON i.country_code = i3.country_code
ORDER BY i.AVG2011 DESC;